In [ ]:
#Imports

In [9]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sqlite3 as lite
import sys

low_memory=False

plt.style.use('fivethirtyeight')
sns.set()
%matplotlib inline
#import dataset
data = pd.read_csv('CandidateSummaryActionSenate.csv')
#trimming unecessary columns
ndata = data[['can_nam','can_off', 'can_par_aff','ind_ite_con','ind_uni_con', 'ind_con'  ]]
ndata.columns=['candidate name' ,'running for', 'party', 'individual itemized', 'individual unitemized', "total individual contributions" ]
#prepare for cleaning

ndata[1:10]






,candidate name,running for,party,individual itemized,individual unitemized,total individual contributions
1,"EHLER, CHARLES JOSEPH",S,REP,NaN,NaN,NaN
2,"MOHARIR, MUKUND MARTAND",S,IND,NaN,NaN,NaN
3,"FRAZIER, RYAN LAVAR",S,REP,"$237,080.00","$19,321.00","$256,401.00"
4,"GRASSLEY, CHARLES E SENATOR",S,REP,"$4,414,642.00","$1,218,408.00","$5,633,053.00"
5,"BLUNT, ROY",S,REP,"$6,444,218.00","$414,311.00","$6,658,523.00"
6,"PEARSON, CECIL WELDON JR",S,DEM,NaN,NaN,NaN
7,"GATES, MICHELLE",S,UNK,NaN,NaN,NaN
8,"BEITIKS, MIKELIS VALAINIS",S,IND,"$2,995.00","$2,639.00","$5,634.00"
9,"BOOZMAN, JOHN",S,REP,"$2,040,027.00","$112,514.00","$2,152,542.00"


In [8]:
plt.style.use('fivethirtyeight')
sns.set()
%matplotlib inline

ddata = pd.read_csv('indivs16a.csv')
ddata = ddata[['Candidate','donor', 'amount']]
ddata[1:10]


,Candidate,donor,amount
1,DELANEY,MARTINMR,25
2,ABRAHAMS,MARSHALL,2700
3,POWERS,MICHAEL,250
4,HUMMEL,ROBERTMR,100
5,HUMMEL,ROBERTMR,100
6,DOLIFKA,LANI,500
7,HUMMEL,ROBERTMR,100
8,WHITE,WARDH,500
9,YOUNGDON,YUN,250
